In [1]:
from concurrent.futures import ProcessPoolExecutor
# from loky import ProcessPoolExecutor  # for Windows users

def parallel(func, iterable):
    e = ProcessPoolExecutor()
    return e.map(func, iterable)

In [66]:
from pathlib import Path
import os
from tqdm import tqdm
from glob import glob
from pathlib import Path
import numpy as np
import torch
from fastai.data.all import *
from fastai.vision.all import *

In [3]:
image_file_extensions = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def is_image_path_valid(path: Path):
    return path.is_file() and path.suffix in image_file_extensions

def load_image_file(path):
    return Image.open(path)

In [10]:
from utils import load_images_recursively

In [7]:
image_dir = Path("./data/new_crops")

In [72]:
fns = get_image_files(image_path / 'before')
failed = verify_images(fns)

In [75]:
def load_image_paths(path: Path):
    fns = get_image_files(path)
    
    return fns

In [76]:
before_paths = load_image_paths(image_path / 'before')
after_paths = load_image_paths(image_path / 'after')

In [35]:
from PIL import Image
import numpy as np

mean_rgb = (131.0912, 103.8827, 91.4953)
image_shape = (224,224,3)

def load_image_for_feature_extraction(path='', shape=image_shape):
    '''
    Referenced from VGGFace2 Paper:
    Q. Cao, L. Shen, W. Xie, O. M. Parkhi, and A. Zisserman, “VGGFace2: A dataset for recognising faces across pose and age,” arXiv:1710.08092 [cs], May 2018
    '''
    short_size = 224.0
    crop_size = shape
    img = Image.open(path)
    im_shape = np.array(img.size)    # in the format of (width, height, *)
    img = img.convert('RGB')

    ratio = float(short_size) / np.min(im_shape)
    img = img.resize(size=(int(np.ceil(im_shape[0] * ratio)),   # width
                           int(np.ceil(im_shape[1] * ratio))),  # height
                     resample=Image.BILINEAR)

    x = np.array(img)  # image has been transposed into (height, width)
    newshape = x.shape[:2]
    h_start = (newshape[0] - crop_size[0])//2
    w_start = (newshape[1] - crop_size[1])//2
    x = x[h_start:h_start+crop_size[0], w_start:w_start+crop_size[1]]
    
    # normalize colors to prevent overfitting on color differences 
    x = x - mean_rgb
    
    # returns transformed image, and original image
    return x

In [25]:
import joblib

lasso_model = joblib.load("./saved_model/lasso.joblib")

In [26]:
from saved_model.prepare_resnet50 import prepare_resnet_model

resnet_model = prepare_resnet_model("./saved_model/resnet50_ft_weight.pkl")

In [32]:
from saved_model.binary_classifier import load_pretrained_classifier

binary_classifier = load_pretrained_classifier('./saved_model/weights-2.pth')

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def full_lime_pipeline(x):
    x = torch.Tensor(x.transpose(0, 3, 1, 2))  # nx3x224x224
    x = x.to(device)
    x = resnet_model(x).detach().cpu().numpy()
    return lasso_model.predict(x)

In [43]:
def full_nn_pipeline(x):
    x = torch.Tensor(x.transpose(0, 3, 1, 2))  # 1x3x224x224
    x = x.to(device)
    x = resnet_model(x)
    x = torch.sigmoid(binary_classifier(x))
    x = torch.round(x)
    return x.detach().cpu().numpy()

In [78]:
import pandas as pd

results = pd.DataFrame({'sample_paths': before_paths})

In [79]:
results['lasso_results_1'] = np.nan
results['nn_results_1'] = np.nan
results['lasso_results_2'] = np.nan
results['nn_results_2'] = np.nan

In [80]:
from tqdm import tqdm
import math

In [81]:
np.expand_dims(load_image_for_feature_extraction(before_paths[0]), 0).shape

(1, 224, 224, 3)

In [82]:
def preprocess(path):
    return np.expand_dims(load_image_for_feature_extraction(path), 0)

for i, (before_path, after_path) in tqdm(enumerate(zip(before_paths, after_paths)), total=len(before_paths)):
    if before_path.stem != after_path.stem:
        print(f"Before and after don't match for index {i}, before: {before_path}, after: {after_path}")
        break
        
    results.loc[i,'lasso_results_1'] = full_lime_pipeline(preprocess(before_path))
    results.loc[i,'lasso_results_2'] = full_lime_pipeline(preprocess(after_path))
    results.loc[i,'nn_results_1'] = full_nn_pipeline(preprocess(before_path)).squeeze()
    results.loc[i,'nn_results_2'] = full_nn_pipeline(preprocess(after_path)).squeeze()

100%|███████████████████████████████████████| 1292/1292 [00:56<00:00, 22.75it/s]


In [83]:
results

,sample_paths,lasso_results_1,nn_results_1,lasso_results_2,nn_results_2
0,data/new_crops/before/haircut face before after 3_89.jpg,1.0,1.0,0.0,1.0
1,data/new_crops/before/makeup before after 3_108.jpg,0.0,0.0,0.0,1.0
2,data/new_crops/before/makeup before after 3_55.jpg,0.0,0.0,0.0,0.0
3,data/new_crops/before/makeup before after arabic_51.jpg,1.0,1.0,1.0,1.0
4,data/new_crops/before/beard before after 2_30.jpg,0.0,0.0,0.0,0.0
...,...,...,...,...,...
1287,data/new_crops/before/makeup before after 3_39.jpg,1.0,0.0,1.0,1.0
1288,data/new_crops/before/glasses before after 1_11.jpg,0.0,0.0,0.0,0.0
1289,data/new_crops/before/beard before after 1_32.jpg,0.0,0.0,0.0,0.0
1290,data/new_crops/before/haircut face before after 1_69.jpg,0.0,0.0,0.0,1.0


In [123]:
total = len(results)
total

1292

In [86]:
lasso_1, lasso_2 = results['lasso_results_1'], results['lasso_results_2']

In [102]:
lasso_flipped = (lasso_1 != lasso_2).sum()
lasso_flipped

200

In [103]:
nn_flipped = (results['nn_results_1'] != results['nn_results_2']).sum()
nn_flipped

428

In [104]:
results['lasso_results_1'].isna().sum()

0

In [105]:
results.to_csv("./results/batch_preds.csv")

In [107]:
lasso_lib_to_con = len(results[(results['lasso_results_1'] == 0) & (results['lasso_results_2'] == 1)])
lasso_lib_to_con

92

In [109]:
lasso_con_to_lib = len(results[(results['lasso_results_1'] == 1) & (results['lasso_results_2'] == 0)])
lasso_con_to_lib

108

In [110]:
nn_lib_to_con = len(results[(results['nn_results_1'] == 0) & (results['nn_results_2'] == 1)])
nn_lib_to_con

231

In [111]:
nn_con_to_lib = len(results[(results['nn_results_1'] == 1) & (results['nn_results_2'] == 0)])
nn_con_to_lib

197

In [124]:
nn_flipped / total

0.33126934984520123